In [5]:
%load_ext autoreload
%autoreload

import serial
import time

import struct
import numpy as np

# Change it to COMx on a MS Windows platform
# serUSB = '/dev/tty.usbmodem1454301'
serUSB = 'COM6'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
with serial.Serial(serUSB, 9600, timeout=1) as ser:
    ser.write(b'101,2')

SerialException: could not open port 'COM6': FileNotFoundError(2, 'The system cannot find the file specified.', None, 2)

In [7]:
import drv_gpio as drv
import dut_func as dut

ser = serial.Serial(serUSB, 9600, timeout=1)
drv.connect(ser)

IndentationError: unindent does not match any outer indentation level (dut_func.py, line 59)

Now for first turn on procedure:

In [2]:
# NOTE: first power on requires physical switch actions on DUT board as well as PIC commands

# FIRST: SWITCH microntroller VDD power ON (i.e. power up microcontroller)
time.sleep(1) # want to delay 1us
drv.gpio_pin_reset(*PIC_PINS['NRESET_FULL_CHIP'])
drv.gpio_pin_reset(*PIC_PINS['NRESET_DPE_ENGINE'])
dut.pads_defaults()
dut.vrefs_off()

Once this is stable, turn on VDD to A0 DPE chip at 5V. 

This means physically activating switch on A0 DUT board.

Turn 3-way switch (S5) to position 1 for board-supplied voltage (see page 1 of schematics)

In [3]:
# Proceed with power on procedure
while True:
    powergoodhigh = drv.gpio_pin_is_high(*PIC_PINS['PWR_GOOD'])
    if powergoodhigh==False: continue
    else:
        time.sleep(100) # want to delay (1ms)
        break

NameError: name 'drv' is not defined

In [ ]:
dut.vrefs_defaults()
# For ADC VREF HI and LO, must physically turn on switches S6 and S7 on board

In [ ]:
# TO BE WRITTEN: Initialize clocks (still to be written in firmware: enable/disable API)
#
#
drv.clk_start('CK_ARRAY')
drv.clk_start('ADC_CK')

time.sleep(10) # want to delay 10us
dut.reset_chip()
time.sleep(1) #delay(1 P_CK_ARRAY clock period)

In [ ]:
# scan_control function during power on
dut.pads_defaults()
drv.gpio_pin_reset(*PIC_PINS['SERIAL_CK_IN'])

# drv.spi_serial_write function address handles the SERIAL_CHAIN_SEL0 and 1 values

# Cat: question for SDL: Does SERIAL_CK_IN need to go high to latch the serial chain select values?
    
# From Cookbook: P_SERIAL_CK_IN number of clock pulses = number of bits read in using P_SERIAL_BUS_IN # Number of clock pulses must exactly equal the number of scan bits being read in 
# for pulse in P_SERIAL_CK_IN clock pulses:
# P_SERIAL_BUS_IN data latched in # Refer to Figure 2 for timing diagram and to Table 3 for recommended initial values to use during power on
# NOTE: Need to check that the ordering lsb to msb is correct from table...
data = bytes([0b00000010, 0b00000001, 0b00100000, 0b00000000,0b00110000, 0b01000000,0b00010000])
drv.spi_serial_write(1, data) # addr 1 here means SERIAL_CHAIN_SEL0 is 1 and _SEL1 is 0


In [ ]:
dut.reset_dpe()

# Identify globally that chip has been powered on
global powered_on
powered_on = 1